In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Sat Jul 11 11:10:46 2020

@author: ADMIN
"""
import keras
from keras.models import Model
from keras.layers import Conv2D,MaxPool2D,Dropout,Dense,Input,concatenate,GlobalAveragePooling2D,AveragePooling2D,Flatten,BatchNormalization
import math
from keras.optimizers import SGD
from keras.utils.vis_utils import plot_model
from keras.datasets import cifar10
import cv2
from keras.utils import np_utils
import numpy as np
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
#---------------------------------------------------------------

from google.colab import drive
drive.mount('/content/drive')

def load_cifar10_data(img_rows, img_cols):
    
    (X_train,Y_train), (X_test, Y_test) = cifar10.load_data()
    
    X_train = np.array([cv2.resize(img, (img_rows,img_cols)) for img in X_train[:,:,:,:]])
    X_test  = np.array([cv2.resize(img, (img_rows,img_cols)) for img in X_test[:,:,:,:]])
    
    classes = np.unique(Y_train)
    nClasses = len(classes)
    X_train.shape, X_test.shape
    Y_train  = to_categorical(Y_train,nClasses)
    Y_test  = to_categorical(Y_test,nClasses)    
    
    X_train = X_train.astype('float16')
    X_test = X_test.astype('float16')
    X_train = X_train / 255.
    X_test = X_test  / 255.
    
    return X_train, Y_train, X_test, Y_test, nClasses
#---------------------------------------------------------------

def inception_module(x,
                     filters_1x1,
                     filters_3x3_reduce,
                     filters_3x3,
                     filters_5x5_reduce,
                     filters_5x5,
                     filters_pool_proj
                     ):
  conv_1x1 = Conv2D(filters_1x1, (1, 1), padding='same', activation='relu', 
            kernel_initializer=kernel_init, bias_initializer=bias_init)(x)
  bn1 = BatchNormalization()(conv_1x1)  
  conv_3x3 = Conv2D(filters_3x3_reduce, (1, 1), padding='same', activation='relu', 
            kernel_initializer=kernel_init, bias_initializer=bias_init)(bn1)
  conv_3x3 = Conv2D(filters_3x3, (3, 3), padding='same', activation='relu', 
            kernel_initializer=kernel_init, bias_initializer=bias_init)(conv_3x3)
  bn3 = BatchNormalization()(conv_3x3)  
  conv_5x5 = Conv2D(filters_5x5_reduce, (1, 1), padding='same', activation='relu', 
            kernel_initializer=kernel_init, bias_initializer=bias_init)(bn1)
  conv_5x5 = Conv2D(filters_5x5, (5, 5), padding='same', activation='relu',
            kernel_initializer=kernel_init, bias_initializer=bias_init)(conv_5x5)
  bn5 = BatchNormalization()(conv_5x5) 
  pool_proj = MaxPool2D((3, 3), strides=(1, 1), padding='same')(x)
  pool_proj = Conv2D(filters_pool_proj, (1, 1), padding='same', activation='relu', 
            kernel_initializer=kernel_init, bias_initializer=bias_init)(pool_proj)
  output = concatenate([conv_1x1, bn3, bn5, pool_proj], axis=3)

  return output
#-----------------------------------------------------------
def write_model(model):
    model_json = model.to_json()
    with open("/content/drive/My Drive/python/modelInceptionAux2_Cifar10 70 30 400.json", "w") as json_file:
       json_file.write(model_json)
    model.save_weights("/content/drive/My Drive/python/model_weightsInceptionAux2_Cifar10 70 30 400.h5")
    print("Saved model to drive")
#------------------------------------------------------------
X_train, y_train, X_test, y_test, nClasses = load_cifar10_data(70,70)

xdata = np.concatenate((X_train, X_test), axis=0)
ydata = np.concatenate((y_train, y_test), axis=0)
X_train,X_test,y_train,y_test = train_test_split(xdata, ydata, test_size=0.2, random_state=30 ,shuffle=True)
X_train.shape,X_test.shape,y_train.shape,y_test.shape

kernel_init = keras.initializers.glorot_uniform()
bias_init = keras.initializers.Constant(value=0.2)

input_layer = Input(shape=(70,70,3))
x = Conv2D(64, (7, 7), padding='same', strides=(2, 2), activation='relu', 
                       kernel_initializer=kernel_init, 
                       bias_initializer=bias_init)(input_layer)

x = BatchNormalization()(x)
x = MaxPool2D((3, 3), padding='same', strides=(2, 2))(x)
x = Conv2D(64, (1, 1), padding='same', strides=(1, 1), activation='relu')(x)
x = BatchNormalization()(x)
x = MaxPool2D((3, 3), padding='same', strides=(2, 2))(x)

x = inception_module(x,
                     filters_1x1=64,
                     filters_3x3_reduce=96,
                     filters_3x3=128,
                     filters_5x5_reduce=16,
                     filters_5x5=32,
                     filters_pool_proj=32)
#-----------------------------------------------------------------
x1 = AveragePooling2D((5, 5), strides=3)(x)
x1 = Conv2D(128, (1, 1), padding='same', activation='relu')(x1)
x1 = BatchNormalization()(x1)
x1 = Flatten()(x1)
x1 = Dense(1024, activation='relu')(x1)
x1 = Dropout(0.7)(x1)
x1 = Dense(10, activation='softmax', name='auxilliary_output_1')(x1)

x = inception_module(x,
                     filters_1x1=112,
                     filters_3x3_reduce=144,
                     filters_3x3=288,
                     filters_5x5_reduce=32,
                     filters_5x5=64,
                     filters_pool_proj=64)
#----------------------------------------------------------------
x2 = AveragePooling2D((5, 5), strides=3)(x)
x2 = Conv2D(128, (1, 1), padding='same', activation='relu')(x2)
x2 = BatchNormalization()(x2)
x2 = Flatten()(x2)
x2 = Dense(1024, activation='relu')(x2)
x2 = Dropout(0.7)(x2)
x2 = Dense(10, activation='softmax', name='auxilliary_output_2')(x2)

x = inception_module(x,
                     filters_1x1=256,
                     filters_3x3_reduce=160,
                     filters_3x3=320,
                     filters_5x5_reduce=32,
                     filters_5x5=128,
                     filters_pool_proj=128
                     #,name='inception_4e'
                     )
#------------------------------------------------------------------
x = GlobalAveragePooling2D(name='avg_pool_5_3x3/1')(x)
x = Dropout(0.4)(x)
x = Dense(512, activation='relu')(x)
x = Dense(10, activation='softmax', name='output')(x)

epochs = 400
initial_lrate = 0.01

sgd = SGD(lr=initial_lrate, momentum=0.9, nesterov=False)

model = Model(input_layer, [x1, x2, x])
model.compile(loss=['categorical_crossentropy', 'categorical_crossentropy', 
                    'categorical_crossentropy'], loss_weights=[0.3, 0.3, 1], 
                     optimizer=sgd, metrics=['accuracy'])
model.summary()
plot_model(model, to_file='/content/drive/My Drive/python/modelCifar10_2Aux C30.png', show_shapes=True, show_layer_names=True)

history = model.fit(X_train, [ y_train, y_train, y_train], 
            validation_data=(X_test, [ y_test, y_test, y_test]),epochs=epochs)
#---------------------------------------------------------------------
save_model = write_model(model)



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 70, 70, 3)    0                                            
__________________________________________________________________________________________________
conv2d_23 (Conv2D)              (None, 35, 35, 64)   9472        input_2[0][0]                    
__________________________________________________________________________________________________
batch_normalization_14 (BatchNo (None, 35, 35, 64)   256         conv2d_23[0][0]                  
__________________________________________________________________________________________________
max_pooling2d_6 (MaxPooling2D)  (None, 18, 18, 64)   0        